# Access to Healthy Assets and Hazards Accessibility Calculations

This notebook produces accessibility calculations from a set of Points of Interest (POIs) to a set of postcodes using a Valhalla routing cluster. It automatically detects the size of the Valhalla cluster and distributes the workload accordingly.

This creates measures for:

* **Retail**
  * Fast Food
  * Gambling Outlets
  * Pubs / Bars etc
  * Leisure Facilities

* **Health**
    * GP Surgeries
    * Pharmacies
    * Hospitals
    * Dentists

* **Blue Space**

The code requires that the AHAH Data Setup notebook has been run to create the input data; and that the AHAH Cluster Setup notebook has been run to create the local Valhalla routing services.
## Imports and Setup

In [13]:
import sys
import os
import geopandas as gpd

In [3]:
# Ensure the 'code' directory is in the system path for module imports
sys.path.insert(0, './code')
from poi_accessibility import POIAccessibilityCalculator

## Simple Example

The following code is the simplest usage with auto-detected cluster:

```python

# Create calculator - auto-detects Valhalla cluster
calc = POIAccessibilityCalculator(
    postcodes_path="data/postcodes/postcodes_geoparquet.parquet",
    results_dir="results/"
)

print(calc)
print(f"Detected {calc.cluster.num_workers} Valhalla workers")
print(f"Worker ports: {calc.cluster.active_ports}")

```

## Parameter Reference

The following table describes all parameters for the `POIAccessibilityCalculator` class.

### Core Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `postcodes_path` | str/Path | *required* | Path to postcodes GeoParquet |
| `results_dir` | str/Path | "./results" | Output directory for GeoParquet results |
| `postcode_id_col` | str | "pcd7" | Column name for postcode identifier |
| `postcode_lsoa_col` | str | "lsoa21cd" | Column for LSOA codes (optional) |
| `postcode_lat_col` | str | "lat" | Column for latitude |
| `postcode_lon_col` | str | "long" | Column for longitude |

### Cluster Detection

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `base_port` | int | 8010 | Starting port for cluster detection |
| `host` | str | "localhost" | Valhalla host address |
| `auto_detect_cluster` | bool | True | Auto-detect cluster size |
| `num_workers` | int | None | Override with fixed worker count |

### Routing Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `batch_size` | int | 50 | Postcodes per Valhalla request |
| `k_nearest` | int | 4 | Nearest POIs to consider per postcode |
| `costing` | str | "auto" | Routing mode: auto, pedestrian, bicycle, etc. |
| `request_timeout` | int | 30 | Timeout per request (seconds) |

## Custom Configuration

The following code shows how to customize the calculator with specific parameters:

```python
# Custom configuration
calc = POIAccessibilityCalculator(
    # Input data
    postcodes_path="data/postcodes/postcodes_geoparquet.parquet",
    results_dir="results/retail/",
    
    # Postcode column names (adjust to match your data)
    postcode_id_col="pcd7",
    postcode_lsoa_col="lsoa21cd",
    postcode_lat_col="lat",
    postcode_lon_col="long",
    
    # Cluster settings
    base_port=8020,          # Starting port to probe
    # num_workers=8,         # Uncomment to override auto-detection
    
    # Routing parameters
    batch_size=50,           # Postcodes per request
    k_nearest=4,             # Consider 4 nearest POIs
    costing="auto",          # Driving mode
    request_timeout=30,      # 30 second timeout
)
```

In [16]:
# Custom configuration
calc = POIAccessibilityCalculator(
    # Input data
    postcodes_path="data/postcodes/postcodes.parquet",
    results_dir="./data/routing_results/",
    
    # Postcode column names (adjust to match your data)
    postcode_id_col="pcd7",
    postcode_lsoa_col="lsoa21cd",
    postcode_lat_col="lat",
    postcode_lon_col="long",
    
    # Cluster settings
    base_port=8020,          # Starting port to probe
        
    # Routing parameters
    batch_size=50,           # Postcodes per request
    k_nearest=4,             # Consider 4 nearest POIs
    costing="auto",          # Driving mode
    request_timeout=30,      # 30 second timeout
)

Detecting Valhalla cluster (probing ports 8020+)...
  ✅ Detected 7 active workers
     Ports: 8020-8026


## Processes All POI

Process multiple POI datasets in sequence:

In [ ]:
# Define POI files to process
poi_files = [
    "./data/retail/fast_food.parquet",
    "./data/retail/gambling.parquet",
    "./data/retail/leisure.parquet",
    "./data/retail/pub_bar.parquet",
    "./data/retail/tobacco.parquet",
    "./data/green_blue/bluespace.parquet",
    "./data/green_blue/greenspace_active.parquet",
    "./data/health/dentist.parquet",
    "./data/health/GP.parquet",
    "./data/health/hospital.parquet",
    "./data/health/pharmacy.parquet",
]

# Process all POIs
results = calc.calculate_batch(
    poi_files,
    resume=False,  # Skip POIs that already have complete results
)

Processing 10 POI datasets...

Processing POI: fast_food.parquet
  Output name: fast_food
  Loaded 53,160 POI locations
  Using 7 Valhalla workers
  Batch size: 50, k_nearest: 4
  Processing 120 postcode prefixes...


Prefixes:   1%|          | 1/120 [00:06<13:04,  6.59s/it]

AB: 17,407 postcodes, 0 missing


Prefixes:   2%|▏         | 2/120 [00:11<10:40,  5.43s/it]

AL: 7,795 postcodes, 0 missing


...

WV: 10,481 postcodes, 0 missing


Prefixes:  99%|█████████▉| 119/120 [10:02<00:04,  4.58s/it]

YO: 19,251 postcodes, 0 missing


Prefixes: 100%|██████████| 120/120 [10:02<00:00,  5.02s/it]


ZE: 655 postcodes, 1 missing

Combining results...

✅ Saved to: data/routing_results/pharmacy.parquet

Summary:
  Total postcodes: 1,745,013
  Missing times: 121
  Mean time: 2.41 min
  Median time: 1.65 min

✅ Completed 10/10 POI datasets


In [ ]:

# Define POI files to process
poi_files = [
    "./data/green_blue/bluespace.parquet",
]

# Process all POIs
results = calc.calculate_batch(
    poi_files,
    resume=False,  # Skip POIs that already have complete results
)

Processing 1 POI datasets...

Processing POI: bluespace.parquet
  Output name: bluespace
  Loaded 8,574,806 POI locations
  Using 7 Valhalla workers
  Batch size: 50, k_nearest: 4
Loading postcodes from data/postcodes/postcodes.parquet...
  Loaded 1,745,013 postcodes
  Processing 120 postcode prefixes...


Prefixes:   1%|          | 1/120 [00:15<31:02, 15.65s/it]

AB: 17,407 postcodes, 0 missing


Prefixes:   2%|▏         | 2/120 [00:24<22:51, 11.62s/it]

AL: 7,795 postcodes, 0 missing


...

WV: 10,481 postcodes, 0 missing


Prefixes:  99%|█████████▉| 119/120 [24:42<00:10, 10.77s/it]

YO: 19,251 postcodes, 0 missing


Prefixes: 100%|██████████| 120/120 [24:44<00:00, 12.37s/it]


ZE: 655 postcodes, 0 missing

Combining results...

✅ Saved to: data/routing_results/bluespace.parquet

Summary:
  Total postcodes: 1,745,013
  Missing times: 130
  Mean time: 1.67 min
  Median time: 1.32 min

✅ Completed 1/1 POI datasets


# Check Missing Routes

In [18]:

# Directory containing the results
results_dir = "./data/routing_results/"

# List all parquet files in the directory
parquet_files = [f for f in os.listdir(results_dir) if f.endswith('.parquet')]

# Loop through each parquet file
for file in parquet_files:
    file_path = os.path.join(results_dir, file)
    gdf = gpd.read_parquet(file_path)
    
    # Print statistics for duration_minutes_missing
    print(f"Statistics for {file}:")
    print(gdf['duration_minutes_missing'].value_counts())
    print("\n")

Statistics for gambling.parquet:
duration_minutes_missing
False    1744832
True         181
Name: count, dtype: int64


Statistics for GP.parquet:
duration_minutes_missing
False    1744893
True         120
Name: count, dtype: int64


Statistics for fast_food.parquet:
duration_minutes_missing
False    1744836
True         177
Name: count, dtype: int64


Statistics for leisure.parquet:
duration_minutes_missing
False    1744832
True         181
Name: count, dtype: int64


Statistics for hospital.parquet:
duration_minutes_missing
False    1744895
True         118
Name: count, dtype: int64


Statistics for pharmacy.parquet:
duration_minutes_missing
False    1744892
True         121
Name: count, dtype: int64


Statistics for greenspace_active.parquet:
duration_minutes_missing
False    1744955
True          58
Name: count, dtype: int64


Statistics for tobacco.parquet:
duration_minutes_missing
False    1744832
True         181
Name: count, dtype: int64


Statistics for pub_bar.parquet:
durati

## LSOA Summary Statistics

Identify LSOAs with missing accessibility values:

In [19]:
# Directory containing the results
results_dir = "./data/routing_results/"

# List all parquet files in the directory
parquet_files = [f for f in os.listdir(results_dir) if f.endswith('.parquet')]

# Loop through each parquet file and print lsoa21cd where 100% of duration_minutes_missing are True
for file in parquet_files:
    file_path = os.path.join(results_dir, file)
    gdf = gpd.read_parquet(file_path)
    print(f"File: {file}")
    if 'lsoa21cd' not in gdf.columns or 'duration_minutes_missing' not in gdf.columns:
        print("  Missing required columns ('lsoa21cd' or 'duration_minutes_missing').\n")
        continue
    all_missing = gdf.groupby('lsoa21cd')['duration_minutes_missing'].all()
    lsoas = all_missing[all_missing].index.tolist()
    if not lsoas:
        print("  No LSOAs with 100% missing durations.\n")
    else:
        print(f"  {len(lsoas)} LSOAs with 100% missing durations:")
        print(f"  {lsoas}\n")


File: gambling.parquet
  No LSOAs with 100% missing durations.

File: GP.parquet
  No LSOAs with 100% missing durations.

File: fast_food.parquet
  No LSOAs with 100% missing durations.

File: leisure.parquet
  No LSOAs with 100% missing durations.

File: hospital.parquet
  No LSOAs with 100% missing durations.

File: pharmacy.parquet
  No LSOAs with 100% missing durations.

File: greenspace_active.parquet
  No LSOAs with 100% missing durations.

File: tobacco.parquet
  No LSOAs with 100% missing durations.

File: pub_bar.parquet
  No LSOAs with 100% missing durations.

File: dentist.parquet
  No LSOAs with 100% missing durations.

File: bluespace.parquet
  No LSOAs with 100% missing durations.



---

## Command Line Usage

The module can also be run from the command line:

```bash
# Single POI
python code/poi_accessibility.py data/postcodes.parquet data/poi/supermarkets.parquet

# Multiple POIs
python code/poi_accessibility.py data/postcodes.parquet \
    data/poi/supermarkets.parquet \
    data/poi/pharmacies.parquet \
    data/poi/gp_surgeries.parquet

# Custom options
python code/poi_accessibility.py data/postcodes.parquet data/poi/banks.parquet \
    --results-dir output/ \
    --batch-size 100 \
    --k-nearest 5 \
    --costing pedestrian

# Override cluster detection
python code/poi_accessibility.py data/postcodes.parquet data/poi/atms.parquet \
    --num-workers 8
```